In [1]:
#importing libraries to be used
import numpy as np
import pandas as pd

In [2]:
#reading dataframes
movies = pd.read_csv('tmdb_5000_movies.csv')
credits = pd.read_csv('tmdb_5000_credits.csv')

In [3]:
#merging dataframes
movies=movies.merge(credits,on='title')

In [4]:
#removing unimportant columns
#for content based recommender system, we need to create tags for each movie

In [5]:
#checking for all present columns
movies.info()

#imp columns:
#genre - people usually like watching movbies of similar genre
#id - for fetching posters from tmdb site while creating our website
#keywords
#title - keeping this for uniformity and removing original title since it can be in any other language which will create inbalance in the data
#overview - content similarity can be very well judged by this
#cast - people usually have preferences based on favourite actor or actress
#crew - people like movies from same directors

#release-date - depending on age or interest people may like movies if a specific time period - try incorporating this later - not now

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4809 entries, 0 to 4808
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4809 non-null   int64  
 1   genres                4809 non-null   object 
 2   homepage              1713 non-null   object 
 3   id                    4809 non-null   int64  
 4   keywords              4809 non-null   object 
 5   original_language     4809 non-null   object 
 6   original_title        4809 non-null   object 
 7   overview              4806 non-null   object 
 8   popularity            4809 non-null   float64
 9   production_companies  4809 non-null   object 
 10  production_countries  4809 non-null   object 
 11  release_date          4808 non-null   object 
 12  revenue               4809 non-null   int64  
 13  runtime               4807 non-null   float64
 14  spoken_languages      4809 non-null   object 
 15  status               

In [6]:
movies=movies[['movie_id','title','overview','genres','keywords','cast','crew']]

In [7]:
#in the preprocessed data we need only three columns - movie_id, title and tags

#tags=overview+genre+keywords+cast(top 3)+crew(director) - to be done after formatting the data in these columns, removing the duplicate date and also the missing data

In [8]:
#checking for missing data
movies.isnull().sum()

movie_id    0
title       0
overview    3
genres      0
keywords    0
cast        0
crew        0
dtype: int64

In [9]:
#removing missing data
movies.dropna(inplace=True)

In [10]:
#checking for duplicate data
movies.duplicated().sum()

0

In [11]:
#checking fir the current format of data to decide the changes to be applied to bring it in the roght format
movies.iloc[0].genres

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [12]:
#ast - module in python to convert string of lists to lists
import ast
ast.literal_eval('[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]')

[{'id': 28, 'name': 'Action'},
 {'id': 12, 'name': 'Adventure'},
 {'id': 14, 'name': 'Fantasy'},
 {'id': 878, 'name': 'Science Fiction'}]

In [13]:
#desired format - only the list of genres/keyword/top 3 actors/directors anmes

#preprocessing
def convert(obj):
    L=[]
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L
#this will not work normally since the string indices are not integers - it is the string of lists - so we first need to convert it to a list and then use the convert function

In [14]:
#preprocessing of genres column - extracting the genre names and storing in a list
#before altering the main column, first check the result of the function so implemented
movies['genres']=movies['genres'].apply(convert)

In [15]:
#similar preprocessing to be applied on keywords column
movies['keywords']=movies['keywords'].apply(convert)

In [16]:
#preprocessing function for extracting names of top three cast
def convert3(obj):
    L=[]
    counter=0
    for i in ast.literal_eval(obj):
        if counter!=3:
            L.append(i['name'])
            counter+=1
        else:
            break
    return L

In [17]:
movies['cast']=movies['cast'].apply(convert3)

In [18]:
#preprocessing function to extract the names of the directors from the crew
def fetch_director(obj):
    L=[]
    for i in ast.literal_eval(obj):
        if i['job']=='Director':
            L.append(i['name'])
            break
    return L

In [19]:
movies['crew']=movies['crew'].apply(fetch_director)

In [20]:
#preprocessing for overview column
movies['overview']=movies['overview'].apply(lambda x:x.split())

In [21]:
#our ultimate aim is to join the columns so formatted to form the tags column for the desired dataset - but for this we need to first remove the sapce is any is oresent in the string which is part of any of the list in these columns
#this is important because there are chances that two people may have the same first name but might differ actually, this will not result in the right  filtering while recommending movies of the same actors or directors - therefore we need to remove these spaces and create a single name which can be unique for everyone

#code for this transformation
movies['genres']=movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])
movies['keywords']=movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
movies['cast']=movies['cast'].apply(lambda x:[i.replace(" ","") for i in x])
movies['crew']=movies['crew'].apply(lambda x:[i.replace(" ","") for i in x])

In [22]:
#combining the data of all the preprocessed columns into the column name tags
movies['tags']=movies['overview']+movies['genres']+movies['keywords']+movies['cast']+movies['crew']

In [23]:
#final preprocessed data
new_df=movies[['movie_id','title','tags']]

In [24]:
new_df['tags']=new_df['tags'].apply(lambda x:" ".join(x))

C:\Users\shrey\AppData\Local\Temp\ipykernel_6152\487797088.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags']=new_df['tags'].apply(lambda x:" ".join(x))


In [25]:
#converting all the data in tags coliumn to lower case for simple comparing
new_df['tags']=new_df['tags'].apply(lambda x:x.lower())

C:\Users\shrey\AppData\Local\Temp\ipykernel_6152\2976555688.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags']=new_df['tags'].apply(lambda x:x.lower())


In [26]:
#stemming of tags to convert the words in it to their root words - avoid repeatition of similar words
import nltk
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()

In [27]:
def stem(text):
    y=[]
    
    for i in text.split():
        y.append(ps.stem(i))
        
    return " ".join(y)

In [28]:
new_df['tags']=new_df['tags'].apply(stem)

C:\Users\shrey\AppData\Local\Temp\ipykernel_6152\3514595201.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags']=new_df['tags'].apply(stem)


In [29]:
#now for comparing two movies, we need to basically compare their tags column which has huge data in textual format and hence comparing it is quite difficukt
#one way in which this can be done is counting the no. of similar words - not a good approach tho
#other way - converting data to vectors(representing points on a cartesian plane) - while recommending we can choose the closest vectors for recommendation - called text vectorization - using bag OfWords technique here
#bagOfWords - combine all tags into a single string - find most common words (top 2k, 5k, 10k....) - make a table of these and then for each movie calculate the no. of occurrences of each word
#new table obtained - each row of this table - called vector
#in vectorization we do not consider stop words - and, are,... (meaningless as per our requirement) (used for sentence formation - do not contribute to the meaning of the sentence)

#this can be done manaually as well - but here we will use the sklearn library which has an inbuilt class and function to perform this text vectorization
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=5000,stop_words='english')

In [30]:
#return scipy pass matrix which is to be converted to an array
vectors=cv.fit_transform(new_df['tags']).toarray()

In [31]:
#now since every movie has been converted to a vector, for recommending similar movies we need to calculate the distance of each movie from every other movie and store this data to find the closest movies later
#euclidian distance - distance from tip of one vector to the tip of other vector
#cosine distance - angle between the vectors 
#we use cosine distance for easy comparison between huge data sets - avoid unnecessary calculations

from sklearn.metrics.pairwise import cosine_similarity

In [32]:
similarity=cosine_similarity(vectors)

In [33]:
similarity[0]

array([1.        , 0.08346223, 0.0860309 , ..., 0.04499213, 0.        ,
       0.        ])

In [34]:
#to find similar movies - sortt he array into reverse order - use enumerate function because we need to keep safe the index otherwise the data will get mixed up
sorted(list(enumerate(similarity[0])),reverse=True,key=lambda x:x[1])[1:6]

[(1216, 0.28676966733820225),
 (2409, 0.26901379342448517),
 (3730, 0.2605130246476754),
 (507, 0.255608593705383),
 (539, 0.2503866978335957)]

In [35]:
#recommendation function to print the names of top 5 similar movies
def recommend(movie):
    movie_index=new_df[new_df['title']==movie].index[0]
    distances=similarity[movie_index]
    movies_list=sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:6]
    
    for i in movies_list:
        print(new_df.iloc[i[0]].title)

In [36]:
#testing of recomemndation function
# recommend('Avatar')

In [37]:
import pickle as pk

In [38]:
pk.dump(new_df.to_dict(),open('movies_dict.pkl','wb'))

In [39]:
pk.dump(similarity,open('similarity.pkl','wb'))